In [3]:
import os
# os.chdir(os.path.dirname(os.path.abspath(__file__))) # change to current file path
import random
import time, shutil
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from hparam import hparam as hp
from data_load import SpeakerDatasetPreprocessed
from speech_embedder_net import SpeechEmbedder, SpeechEmbedder_Softmax, GE2ELoss, GE2ELoss_, get_centroids, get_cossim
from torch.utils.tensorboard import SummaryWriter
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve
import numpy as np
from numpy.linalg import solve
import scipy.linalg
import scipy.stats
from tqdm import tqdm
from utils import compute_eer

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

def get_n_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])

    return params

# Threshold based

In [ ]:
device = torch.device(hp.device)
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

test_dataset = SpeakerDatasetPreprocessed()

test_loader = DataLoader(test_dataset, batch_size=hp.test.N, shuffle=True, num_workers=hp.test.num_workers, drop_last=True)

try:
    embedder_net = SpeechEmbedder().to(device)
    embedder_net.load_state_dict(torch.load(model_path))
except:
    embedder_net = SpeechEmbedder_Softmax(num_classes=5994).to(device)
    embedder_net.load_state_dict(torch.load(model_path))
embedder_net.eval()

print("Number of params: ", get_n_params(embedder_net))

# O2U-Net